In [1]:
from datetime import datetime, timedelta, timezone
import pandas as pd
import numpy as np
import os
import optuna
import logging
import xgboost as xgb
import sys
import joblib
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

"""## Variables"""

current_storage = os.path.join("d:", "data")
vn30f_storage = os.path.join(current_storage, 'vn30f')

start_time = 0
now_time = 9999999999
symbol = 'VN30F1M'
rolling_window = 1 # Số phút muốn dự đoán tiếp theo

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "btc1-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

"""## Formulas"""

def choose_position(roi, trade_threshold = 0.0001):
    pos =0
    # Predict position base on change in future
    if roi > trade_threshold:
        pos = 1
    elif roi < -trade_threshold:
        pos = -1
    else:
        pos = 0

    return pos

def backtest_position_ps(position, price, percentage, periods):
    # Shift positions to align with future price changes and handle NaN by filling with 0
    pos = pd.Series(position, index=pd.Series(price).index).shift(1).fillna(0)
    pos = pd.Series(pos).rolling(periods).sum() #pos for 10 hour predict

    price_array = pd.Series(price).shift(1).fillna(0)

    pos_diff = pos.diff()
    fee = pos_diff*price_array*0.05*0.01

    # Calculate price changes over the given periods
    ch = pd.Series(price) - price_array

    # Calculate total PnL
    total_pnl = pos*ch - fee
    return total_pnl

def calculate_sharpe_ratio(pnl):
    pnl = np.diff(pnl)
    std = np.std(pnl) if np.std(pnl) != 0 else 0.001
    sharpe = np.mean(pnl)/std*np.sqrt(252)
    return sharpe

def sharpe_for_vn30f(y_pred, y_price, trade_threshold, fee_perc, periods):

    # Predict position base on change in future
    pos = [choose_position(roi, trade_threshold) for roi in y_pred]
    pos = np.array(pos)

    # Calculate PNL
    pnl = backtest_position_ps(pos, y_price, fee_perc, periods)
    pnl = np.cumsum(pnl)

    # Standardalize PNL to date
    daily_pnl = [pnl.iloc[i] for i in range(0, len(pnl), 241)]
    daily_pnl = pd.Series(daily_pnl).fillna(0)

    # Calculate Sharpe
    sharpe = calculate_sharpe_ratio(daily_pnl)

    return pos, pnl, daily_pnl, sharpe

def calculate_hitrate(pos_predict, pos_true):
    if len(pos_predict) != len(pos_true):
        raise ValueError("Độ dài của hai mảng không khớp")

    # Tính số lượng dự đoán đúng (các phần tử tương ứng giống nhau)
    correct_predictions = np.sum(pos_predict == pos_true)

    # Tính tỷ lệ hit rate
    hit_rate_value = correct_predictions / len(pos_predict)

    return hit_rate_value

"""# Function for data"""

def scale_data(data):
    scaler = StandardScaler()
    data = np.where(np.isinf(data), np.nan, data)
    data = pd.DataFrame(data)
    data = data.fillna(0)
    scaler.fit(data)
    data=pd.DataFrame(scaler.transform(data), index=data.index, columns=data.columns)

    return data

def split_data(data):
    """
    Hàm này chia dữ liệu thành 2 phần: tập huấn luyện và tập hold out.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu tập huấn luyện.
    pandas.DataFrame: DataFrame chứa dữ liệu tập giữ lại.
    """
    # Chia dữ liệu thành 3 phần
    new_part = np.array_split(data, 3)

    # Access each part individually
    hold_out = new_part[2]
    train_data = pd.concat([new_part[0], new_part[1]], axis=0)

    return train_data, hold_out

def split_optuna_data(data):
    """
    Hàm này chia dữ liệu thành các tập train và test để sử dụng trong quá trình tối ưu hóa bằng Optuna.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu train (đã được chuẩn hóa).
    pandas.DataFrame: DataFrame chứa dữ liệu test (đã được chuẩn hóa).
    pandas.Series: Series chứa nhãn tương ứng với dữ liệu train.
    pandas.Series: Series chứa nhãn tương ứng với dữ liệu test.
    """
    # Chia dữ liệu thành tập train và tập hold out
    train_data, _ = split_data(data)

    # Loại bỏ các cột không cần thiết
    if 'Open' in train_data.columns:
        optuna_data = train_data.drop(['Open','High','Low','Close','Volume', 'Return', 'Date', 'time'], axis=1)
    else:
        optuna_data = train_data.drop(['Close', 'Return', 'Date', 'time'], axis=1)

    # Chuẩn hóa dữ liệu
    optuna_data = scale_data(optuna_data)

    # Chia dữ liệu thành tập train và tập test
    X_train, X_valid, y_train, y_valid = train_test_split(optuna_data, train_data['Return'], test_size=0.5, shuffle=False)

    return X_train, X_valid, y_train, y_valid, train_data

data = pd.read_csv('/kaggle/input/new-data/save_data.csv')
data = data[data.Date >= '2021']
def objective(trial, X_train, X_valid, y_train, y_valid, y_price):

    # Select features based on Optuna's suggestions
    selected_features = []

    at_least_one_feature = False

    for col in X_train.columns:
        select_feature = trial.suggest_categorical(col, [0, 1])
        if select_feature:
            selected_features.append(col)
            at_least_one_feature = True

    # If no feature was selected, force selection of at least one feature
    if not at_least_one_feature:
        # Randomly select one feature to be included
        forced_feature = trial.suggest_categorical('forced_feature', X_train.columns.tolist())
        selected_features.append(forced_feature)

    for t in trial.study.trials:
        if t.state != optuna.trial.TrialState.COMPLETE:
            continue
        if t.params == trial.params:
            return np.nan # t.values  # Return the previous value without re-evaluating i

    trade_threshold  = 0.0001

    # Use only the selected features in training
    X_train_selected = X_train[selected_features]
    X_valid_selected = X_valid[selected_features]

    # Train the model
    model = GradientBoostingRegressor(n_estimators=300)
    model.fit(X_train_selected, y_train)

    y_pred_train = model.predict(X_train_selected)
    y_pred_valid = model.predict(X_valid_selected)

    _, pos_is, _, sharpe_is = sharpe_for_vn30f(y_pred_train, y_price[:len(y_pred_train)], trade_threshold=trade_threshold, fee_perc=0.01, periods=10)
    _, pos_os, _, sharpe_oos = sharpe_for_vn30f(y_pred_valid, y_price[len(y_pred_train):], trade_threshold=trade_threshold, fee_perc=0.01, periods=10)

    print('Trade times in sample:',len(pos_is[pos_is != 0]), 'Trade times out sample:',len(pos_os[pos_os != 0]))

    return sharpe_oos, abs((abs(sharpe_is / sharpe_oos))-1)

X_train, X_valid, y_train, y_valid, train_data = split_optuna_data(data)
"""## Define number of trials (no 2)"""

# Create a study object and optimize the objective function
study = optuna.create_study(directions=['maximize', 'minimize'])
unique_trials = 1000

while unique_trials > len(set(str(t.params) for t in study.trials)):
    study.optimize(lambda trial: objective(trial, X_train, X_valid, y_train, y_valid, train_data['Close']), n_trials=1)
    study.trials_dataframe().fillna(0).sort_values('values_0').to_csv('xgb_feature_trials.csv')
    joblib.dump(study, 'xgbmodel.pkl')

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
[I 2024-07-04 13:35:07,799] A new study created in memory with name: no-name-682c5b00-9a57-4687-98e8-648795a2200b


A new study created in memory with name: no-name-682c5b00-9a57-4687-98e8-648795a2200b


[I 2024-07-04 13:36:06,138] Trial 0 finished with values: [-0.4535726878124157, 10.577015397099332] and parameters: {0: 0, 1: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 0, 8: 1, 9: 1, 10: 1, 11: 0, 12: 1, 13: 0, 14: 1}. 


Trade times in sample: 69703 Trade times out sample: 69631
Trial 0 finished with values: [-0.4535726878124157, 10.577015397099332] and parameters: {0: 0, 1: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0, 7: 0, 8: 1, 9: 1, 10: 1, 11: 0, 12: 1, 13: 0, 14: 1}. 


[I 2024-07-04 13:37:13,529] Trial 1 finished with values: [-0.4285258813263512, 12.117743180103163] and parameters: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 0, 14: 1}. 


...

Trial 2246 failed with value nan.


/tmp/ipykernel_18/2646646429.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  return sharpe_oos, abs((abs(sharpe_is / sharpe_oos))-1)
[I 2024-07-05 01:14:12,634] Trial 2247 finished with values: [0.0, inf] and parameters: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0}. 


Trade times in sample: 65076 Trade times out sample: 10
Trial 2247 finished with values: [0.0, inf] and parameters: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0}. 


[I 2024-07-05 01:15:02,778] Trial 2248 finished with values: [1.4294640796365117, 2.1208757414583315] and parameters: {0: 1, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 1, 10: 0, 11: 0, 12: 1, 13: 1, 14: 0}. 


Trade times in sample: 69703 Trade times out sample: 69693
Trial 2248 finished with values: [1.4294640796365117, 2.1208757414583315] and parameters: {0: 1, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 1, 10: 0, 11: 0, 12: 1, 13: 1, 14: 0}. 


In [2]:
study.trials_dataframe().fillna(0).sort_values('values_0')

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_0,params_1,params_2,params_3,...,params_7,params_8,params_9,params_10,params_11,params_12,params_13,params_14,system_attrs_nsga2:generation,state
2214,2214,-1.803041,1.704169,2024-07-05 01:08:51.132061,2024-07-05 01:09:45.334050,0 days 00:00:54.201989,1,0,0,1,...,0,0,1,0,1,1,1,1,19,COMPLETE
562,562,-1.771719,1.866229,2024-07-04 18:47:17.612895,2024-07-04 18:48:11.747540,0 days 00:00:54.134645,1,0,0,1,...,0,0,1,1,1,1,1,1,9,COMPLETE
210,210,-1.725219,1.923940,2024-07-04 15:51:00.579268,2024-07-04 15:52:03.349671,0 days 00:01:02.770403,0,1,0,1,...,1,0,1,0,1,1,0,1,3,COMPLETE
512,512,-1.717769,1.936622,2024-07-04 18:27:25.770777,2024-07-04 18:28:27.551495,0 days 00:01:01.780718,0,1,0,1,...,1,0,1,0,1,1,0,1,8,COMPLETE
404,404,-1.660341,1.938400,2024-07-04 17:38:53.678820,2024-07-04 17:39:47.091977,0 days 00:00:53.413157,0,0,0,0,...,0,0,1,1,1,1,1,1,6,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,779,2.306936,1.274578,2024-07-04 20:00:17.634590,2024-07-04 20:00:41.851424,0 days 00:00:24.216834,0,0,0,0,...,0,0,1,1,1,0,1,0,11,COMPLETE
1300,1300,2.347021,1.325475,2024-07-04 21:49:44.812285,2024-07-04 21:50:20.963128,0 days 00:00:36.150843,0,0,0,1,...,0,0,1,1,1,1,1,0,14,COMPLETE
1383,1383,2.366969,1.305876,2024-07-04 22:11:01.252632,2024-07-04 22:11:36.380930,0 days 00:00:35.128298,1,0,0,1,...,0,0,1,1,1,1,1,0,15,COMPLETE
879,879,2.556940,1.253893,2024-07-04 20:23:37.678963,2024-07-04 20:24:13.175407,0 days 00:00:35.496444,1,0,0,1,...,0,0,1,1,0,1,1,0,12,COMPLETE
